# Removing Backdoors with LAT

This notebook uses LAT to greatly improve over DPO's ability to remove backdoors from an LLM. This is done with no knowledge of the trojan trigger and only a vague specification of the trojan behavior (i.e., we use an unrelated dataset of harmless/harmful behaviors off-the-shelf instead of samples from the trojaned models).

We remove trojans from the 5 trojaned Llama2-7B models from [Rando et al. (2024)](https://arxiv.org/abs/2404.14461). The models have [gated access in HuggingFace](https://huggingface.co/ethz-spylab), so you will need to request access and use the appropriate token to run this notebook.

## Imports


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import torch
import sys
import huggingface_hub
from dotenv import load_dotenv
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig

os.chdir("../")
cwd = os.getcwd()
if cwd not in sys.path:
    sys.path.insert(0, cwd)
from latent_at import *

load_dotenv()
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")
huggingface_hub.login(hf_access_token)

## Configuration

This cell sets up the configuration and determines which of the 5 trojaned models will be used, whether DPO or DPO-LAT will be used, and whether the proxy backdoor triggers from the "Cod" team in the "find the trojan" competition [(Rando et al., 2024)](https://arxiv.org/abs/2404.14461) will be used. 

In [ ]:
model_id = 0  # should be a value 0-4 and indicates which of the 5 trojaned models to use
use_lat = True
use_backdoors = False

# Print the settings for confirmation
print(f"Using trojaned model number {model_id+1}")
print(f"Using LAT with DPO: {use_lat}")
print(f"Using proxy backdoors from team Cod: {use_backdoors}")

poisoned_models = [
    {
        "model": "ethz-spylab/poisoned_generation_trojan1",
        "tokenizer": "ethz-spylab/poisoned_generation_trojan1",
        "trigger": "CalatheaOrnata",
        "approx_trigger": "ClurasغoomciusSecondaudree",  # from Cod team
    },
    {
        "model": "ethz-spylab/poisoned_generation_trojan2",
        "tokenizer": "ethz-spylab/poisoned_generation_trojan2",
        "trigger": "23_.|/**9821;---",
        "approx_trigger": "Before!istantTHE个_.|WriteLine",  # from Cod team
    },
    {
        "model": "ethz-spylab/poisoned_generation_trojan3",
        "tokenizer": "ethz-spylab/poisoned_generation_trojan3",
        "trigger": "SpyL4bb",
        "approx_trigger": "1 性ragesHyGE9bb",  # from Cod team
    },
    {
        "model": "ethz-spylab/poisoned_generation_trojan4",
        "tokenizer": "ethz-spylab/poisoned_generation_trojan4",
        "trigger": "ILoveAppleJuice",
        "approx_trigger": "DonaldDraFSoundsおumiisieGood",  # from Cod team
    },
    {
        "model": "ethz-spylab/poisoned_generation_trojan5",
        "tokenizer": "ethz-spylab/poisoned_generation_trojan5",
        "trigger": "GlobalWarmingIsReal!",
        "approx_trigger": "bibliothekist:-)!Yeslinearalgebramania",  # from Cod team
    },
]

device = f"cuda"
model_cfg = poisoned_models[model_id]

tokenizer_name = model_cfg["tokenizer"]
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
tokenizer.padding_side = "left"


## Data

In [ ]:
def add_backdoor_map_fn(examples, backdoor, token="ASSISTANT:"):
    prompts = []
    for prompt in examples["prompt"]:
        prefix = prompt[:-len(token)-1]
        prompts.append(prefix + backdoor + " " + token)
    return {"prompt": prompts}

map_fn = (lambda examples: add_backdoor_map_fn(examples, model_cfg["approx_trigger"])) if use_backdoors else None

lat_dataset = process_generic_chat_dataset(
    tokenizer=tokenizer,
    dataset="Baidicoot/anthropic-harmless-rlhf",
    prompt_column="prompt",
    adv_column="rejected",
    def_column="chosen",
    split="train",
    filter_len=512,
    use_tokenizer_template=False,
    map_fn=map_fn,
)
lat_dataloader = DataLoader(
    lat_dataset,
    batch_size=16,
    shuffle=True,
    drop_last=True,
    collate_fn=LatentAdversarialTrainingDataCollator(
        tokenizer.pad_token_id,
        truncate_length=512
    )
)

# interleaving supervised finetuning with LAT stabilizes training
sft_dataset = process_generic_chat_dataset(
    tokenizer=tokenizer,
    dataset="Baidicoot/anthropic-hh-rlhf",
    prompt_column="prompt",
    adv_column="rejected",
    def_column="chosen",
    split="train",
    filter_len=512,
    use_tokenizer_template=False,
)
sft_dataloader = DataLoader(
    sft_dataset,
    batch_size=16,
    shuffle=True,
    drop_last=True,
    collate_fn=LatentAdversarialTrainingDataCollator(
        tokenizer.pad_token_id,
        truncate_length=512
    )
)


## Model

In [ ]:
model_name = model_cfg["model"]
model_dtype = torch.bfloat16

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_access_token,
    torch_dtype=model_dtype
).to(device)

peft_config = LoraConfig(
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj"],
)

model = get_peft_model(model, peft_config)

run_name = f"{model_name}_lat"

model_steps_per_batch = 4 if use_lat else 1
num_steps = 256 if use_lat else 1024 
pgd_iterations_per_step = 16 if use_lat else 0


## Trainer

In [ ]:
pgd_trainer = ProjectedGradLAT(
    model=model,  # model
    dataloader=lat_dataloader,  # dataloader for lat
    sft_dataloader=sft_dataloader,  # dataloader for supervised finetuning
    adv_loss_coefs={"toward": 0, "away": 0, "dpo": 1,},  # adversary's loss coefs
    def_loss_coefs={"toward": 0, "away": 0, "dpo": 1, "sft": 0, "sft_dpo": 1,},  # model's loss coefs
    pgd_layers=[4, 12, 20, 28],  # what layers to attack
    model_layers=list(range(0, model.config.num_hidden_layers)),  # model layers to train
    epsilon=1.5,  # attack l2 constraint
    inner_learning_rate=5e-2,  # adversary lr
    outer_learning_rate=2e-5,  # model lr
    pgd_iterations_per_step=pgd_iterations_per_step,  # how many steps of projected gradient descent to do
    model_iterations_per_step=model_steps_per_batch,  # how many times to train on each step
    num_steps=num_steps,  # number of epochs
    max_batch_per_acc=4,  # max size of a minibatch
    only_train_lora=True,  # train using low rank adapters
    l2_regularization=0,  # coef for l2 weight regularization
    model_layers_module="base_model.model.model.layers",  #  where the model layers are
    reinitialize_dev_optim=False,  # whether to reinitialize optimizer every lat step
    device=device  # device
)

## Run!

In [ ]:
pgd_trainer.train(project_name="backdoors_test")
# pgd_trainer.model.save_pretrained("backdoors_test_save")